# Aggregating and Combining `pandas` DataFrames

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging, joining, and concatinating

## Set Up

Surprise, surprise... we're still working with the Austin Animal Center Data! Let's start with Outcomes

In [1]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
outcomes = pd.read_csv('../data/Austin_Animal_Center_Outcomes_022822.csv', parse_dates=['DateTime', 'Date of Birth'])

In [3]:
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


In [4]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137097 entries, 0 to 137096
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Animal ID         137097 non-null  object        
 1   Name              96095 non-null   object        
 2   DateTime          137097 non-null  datetime64[ns]
 3   MonthYear         137097 non-null  object        
 4   Date of Birth     137097 non-null  datetime64[ns]
 5   Outcome Type      137073 non-null  object        
 6   Outcome Subtype   62653 non-null   object        
 7   Animal Type       137097 non-null  object        
 8   Sex upon Outcome  137095 non-null  object        
 9   Age upon Outcome  137092 non-null  object        
 10  Breed             137097 non-null  object        
 11  Color             137097 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 12.6+ MB


In [6]:
# Let's create our Age in Days column
outcomes['Calculated Age in Days'] = pd.to_datetime(outcomes['DateTime'].dt.date) - outcomes['Date of Birth']

In [8]:
# Grab just the integer here...
outcomes['Calculated Age in Days'] = outcomes['Calculated Age in Days'].dt.days

In [9]:
# Sanity check
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [12]:
# Just using groupby outputs some weird GroupBy object... not helpful
outcomes.groupby('Animal Type')

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is `groups`.

In [14]:
outcomes[outcomes['Animal Type'] == 'Bird']

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
206,A720727,Rooster 11,2016-03-08 13:47:00,Mar 2016,2015-02-14,Adoption,NaN,Bird,Intact Male,1 year,Chicken Mix,Black/Red,388
534,A720734,Rooster 18,2016-03-08 15:07:00,Mar 2016,2015-02-14,Adoption,NaN,Bird,Intact Male,1 year,Chicken Mix,Black/Chocolate,388
985,A779213,NaN,2018-09-07 11:17:00,Sep 2018,2017-08-27,Adoption,Foster,Bird,Unknown,1 year,Quaker Mix,Green/Silver,376
1027,A760051,NaN,2017-10-20 12:40:00,Oct 2017,2016-10-11,Adoption,NaN,Bird,Unknown,1 year,Quaker,Green/Gray,374
1284,A790172,NaN,2019-03-07 00:00:00,Mar 2019,2018-03-05,Euthanasia,Suffering,Bird,Unknown,1 year,Chicken Mix,White/Red,367
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136331,A851000,A851000,2022-02-03 16:39:00,Feb 2022,2020-02-03,Euthanasia,Suffering,Bird,Unknown,2 years,Pigeon,Gray/White,731
136332,A850997,A850997,2022-02-03 16:38:00,Feb 2022,2020-02-03,Euthanasia,Suffering,Bird,Unknown,2 years,Pigeon,Gray/White,731
136376,A850996,Yukon Jack,2022-02-04 09:52:00,Feb 2022,2019-02-03,Transfer,Partner,Bird,Intact Male,3 years,Falcon,Brown/White,1097
136463,A851114,NaN,2022-02-09 15:32:00,Feb 2022,2021-02-06,Transfer,Partner,Bird,Intact Male,1 year,Chicken,Tricolor/White,368


In [13]:
# This returns each group indexed by the group name, e.g. 'Bird',
# along with the row indices of each value
outcomes.groupby('Animal Type').groups

{'Bird': [206, 534, 985, 1027, 1284, 1310, 2220, 2258, 2274, 2417, 2521, 2598, 2712, 2778, 3178, 3379, 3648, 3743, 4003, 4024, 4288, 4702, 4766, 4998, 5063, 5205, 5436, 5656, 5848, 6087, 6236, 6340, 6592, 6682, 7033, 7352, 7428, 7985, 8048, 8315, 8331, 8414, 8538, 8922, 9203, 9448, 9758, 9825, 10103, 10165, 10407, 10657, 10736, 11386, 11616, 11674, 11732, 11765, 11771, 11837, 12205, 12418, 12423, 12451, 12474, 12713, 12902, 12978, 13057, 13063, 13095, 13272, 13317, 13323, 13435, 13474, 13677, 13934, 13950, 13963, 13981, 14108, 14131, 14146, 14193, 15114, 15193, 15543, 15553, 15813, 16022, 16197, 16499, 16866, 17173, 17338, 17390, 17426, 18319, 18359, ...], 'Cat': [0, 4, 7, 8, 10, 11, 14, 15, 16, 17, 18, 20, 24, 26, 34, 37, 49, 54, 56, 66, 67, 68, 70, 75, 78, 80, 83, 84, 89, 90, 92, 94, 95, 97, 98, 102, 113, 115, 116, 117, 118, 120, 122, 126, 139, 141, 142, 145, 147, 148, 151, 152, 156, 157, 158, 164, 167, 168, 170, 171, 176, 178, 184, 191, 192, 194, 200, 202, 203, 207, 209, 212, 215, 2

In [16]:
# Same goes for multi-index groupbys
animal_outcome = outcomes.groupby(  ['Animal Type', 'Outcome Type'] )

In [19]:
# .groups outputs a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

dict_keys([('Cat', 'Rto-Adopt'), ('Dog', 'Adoption'), ('Other', 'Euthanasia'), ('Cat', 'Transfer'), ('Cat', 'Adoption'), ('Cat', 'Return to Owner'), ('Dog', 'Return to Owner'), ('Dog', 'Transfer'), ('Cat', 'Euthanasia'), ('Other', 'Adoption'), ('Dog', 'Rto-Adopt'), ('Cat', 'Died'), ('Dog', 'Euthanasia'), ('Other', 'Transfer'), ('Bird', 'Adoption'), ('Other', 'Disposal'), ('Other', 'Died'), ('Dog', 'Died'), ('Cat', 'Disposal'), ('Other', 'Return to Owner'), ('Bird', 'Euthanasia'), ('Bird', 'Transfer'), ('Livestock', 'Return to Owner'), ('Dog', 'Missing'), ('Other', 'Relocate'), ('Dog', nan), ('Livestock', 'Adoption'), ('Bird', 'Return to Owner'), ('Dog', 'Disposal'), ('Cat', 'Missing'), ('Bird', 'Disposal'), ('Bird', 'Died'), ('Other', 'Missing'), ('Other', 'Rto-Adopt'), ('Bird', 'Relocate'), ('Bird', 'Missing'), ('Other', nan), ('Livestock', 'Transfer'), ('Cat', 'Relocate'), ('Cat', nan), ('Livestock', 'Died'), ('Livestock', 'Euthanasia')])

In [20]:
# We can then get a specific group, such as cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
7,A689724,*Donatello,2014-10-18 18:52:00,Oct 2014,2014-08-01,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,78
8,A680969,*Zeus,2014-08-05 16:59:00,Aug 2014,2014-06-03,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby,63
20,A730621,*Liza,2016-09-10 18:59:00,Sep 2016,2016-05-18,Adoption,NaN,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Calico,115
26,A801106,NaN,2019-08-16 14:05:00,Aug 2019,2019-05-06,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair,Orange Tabby,102
54,A792258,Vesper,2019-04-10 20:53:00,Apr 2019,2016-09-08,Adoption,NaN,Cat,Spayed Female,2 years,Domestic Shorthair Mix,Tortie,944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137072,A846689,Coco Chanel,2022-02-26 17:23:00,Feb 2022,2021-08-19,Adoption,NaN,Cat,Spayed Female,6 months,Domestic Shorthair,Blue Tabby,191
137073,A845330,Mitzi,2022-02-26 18:09:00,Feb 2022,2021-01-28,Adoption,NaN,Cat,Spayed Female,1 year,Domestic Shorthair,Torbie/White,394
137088,A851184,*Papaya,2022-02-28 11:38:00,Feb 2022,2021-02-08,Adoption,NaN,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Orange Tabby/White,385
137090,A847804,*Mahalia,2022-02-28 11:42:00,Feb 2022,2011-12-08,Adoption,NaN,Cat,Spayed Female,10 years,Domestic Shorthair Mix,Brown Tabby/White,3735


## Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [22]:
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


In [21]:
outcomes.groupby('Animal Type').count()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
Animal Type,,,,,,,,,,,,
Bird,636,145,636,636,636,636,380,636,636,636,636,636
Cat,52092,30380,52092,52092,52092,52088,31815,52092,52091,52092,52092,52092
Dog,77091,64516,77091,77091,77091,77076,24590,77089,77089,77091,77091,77091
Livestock,25,3,25,25,25,25,19,25,25,25,25,25
Other,7253,1051,7253,7253,7253,7248,5849,7253,7251,7253,7253,7253


## Exercise

Use `.groupby()` to find the most recent birth date of each (main) animal type.


In [23]:
# Your code here
outcomes.groupby('Animal Type')['Date of Birth'].max()

Animal Type
Bird        2022-01-06
Cat         2022-02-18
Dog         2022-02-14
Livestock   2020-05-28
Other       2022-02-11
Name: Date of Birth, dtype: datetime64[ns]

In [24]:
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


In [29]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137097 entries, 0 to 137096
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Animal ID               137097 non-null  object        
 1   Name                    96095 non-null   object        
 2   DateTime                137097 non-null  datetime64[ns]
 3   MonthYear               137097 non-null  object        
 4   Date of Birth           137097 non-null  datetime64[ns]
 5   Outcome Type            137073 non-null  object        
 6   Outcome Subtype         62653 non-null   object        
 7   Animal Type             137097 non-null  object        
 8   Sex upon Outcome        137095 non-null  object        
 9   Age upon Outcome        137092 non-null  object        
 10  Breed                   137097 non-null  object        
 11  Color                   137097 non-null  object        
 12  Calculated Age in Days  137097

In [31]:
outcomes.groupby(['Outcome Type', 'Sex upon Outcome']).agg('mean')

Calculated Age in Days  \
Outcome Type    Sex upon Outcome                           
Adoption        Intact Female                 419.505568   
                Intact Male                   476.596774   
                Neutered Male                 651.126203   
                Spayed Female                 648.050078   
                Unknown                       389.964286   
Died            Intact Female                 349.628049   
                Intact Male                   304.392157   
                Neutered Male                1859.732673   
                Spayed Female                2099.905263   
                Unknown                       311.192737   
Disposal        Intact Female                 407.277778   
                Intact Male                   735.935484   
                Neutered Male                1987.777778   
                Spayed Female                2470.166667   
                Unknown                       447.166016   
Euthanasia      Intact Female                1133.424632   
                Intact Male                   902.476300   
                Neutered Male                2226.837838   
                Spayed Female                2313.677966   
                Unknown                       503.333588   
Missing         Intact Female                 266.800000   
                Intact Male                   340.846154   
                Neutered Male                1188.285714   
                Spayed Female                1262.733333   
                Unknown                       169.250000   
Relocate        Intact Female                 732.000000   
                Neutered Male                1105.000000   
                Spayed Female                 495.000000   
                Unknown                       612.200000   
Return to Owner Intact Female                1067.053879   
                Intact Male                  1110.232182   
                Neutered Male                1634.609964   
                Spayed Female                1751.059214   
                Unknown                       764.225275   
Rto-Adopt       Intact Female                1481.142857   
                Intact Male                  1503.175000   
                Neutered Male                1254.399577   
                Spayed Female                1252.076923   
                Unknown                      1590.000000   
Transfer        Intact Female                 409.499911   
                Intact Male                   351.252856   
                Neutered Male                1126.408920   
                Spayed Female                1097.636076   
                Unknown                       169.775301   

                                  Calculated Age in Days 2  
Outcome Type    Sex upon Outcome                            
Adoption        Intact Female                   419.505568  
                Intact Male                     476.596774  
                Neutered Male                   651.126203  
                Spayed Female                   648.050078  
                Unknown                         389.964286  
Died            Intact Female                   349.628049  
                Intact Male                     304.392157  
                Neutered Male                  1859.732673  
                Spayed Female                  2099.905263  
                Unknown                         311.192737  
Disposal        Intact Female                   407.277778  
                Intact Male                     735.935484  
                Neutered Male                  1987.777778  
                Spayed Female                  2470.166667  
                Unknown                         447.166016  
Euthanasia      Intact Female                  1133.424632  
                Intact Male                     902.476300  
                Neutered Male                  2226.837838  
                Spayed Female                  2313.6

In [41]:
type_and_color = outcomes.groupby(['Animal Type', 'Color']).count()

In [42]:
type_and_color

Animal ID  Name  DateTime  MonthYear  \
Animal Type Color                                                   
Bird        Black                   44     4        44         44   
            Black/Brown              6     1         6          6   
            Black/Chocolate          5     5         5          5   
            Black/Cream              5     5         5          5   
            Black/Gold               1     1         1          1   
...                                ...   ...       ...        ...   
Other       White/Tricolor          17    13        17         17   
            White/White              7     3         7          7   
            White/Yellow             1     0         1          1   
            Yellow                   1     0         1          1   
            Yellow/Black             2     1         2          2   

                             Date of Birth  Outcome Type  Outcome Subtype  \
Animal Type Color                                                           
Bird        Black                       44            44               34   
            Black/Brown                  6             6                4   
            Black/Chocolate              5             5                0   
            Black/Cream                  5             5                2   
            Black/Gold                   1             1                0   
...                                    ...           ...              ...   
Other       White/Tricolor              17            17                8   
            White/White                  7             7                4   
            White/Yellow                 1             1                0   
            Yellow                       1             1                0   
            Yellow/Black                 2             2                1   

                             Sex upon Outcome  Age upon Outcome  Breed  \
Animal Type Color                                                        
Bird        Black                          44                44     44   
            Black/Brown                     6                 6      6   
            Black/Chocolate                 5                 5      5   
            Black/Cream                     5                 5      5   
            Black/Gold                      1                 1      1   
...                                       ...               ...    ...   
Other       White/Tricolor                 17                17     17   
            White/White                     7                 7      7   
            White/Yellow                    1                 1      1   
            Yellow                          1                 1      1   
            Yellow/Black                    2                 2      2   

                             Calculated Age in Days  
Animal Type Color                                    
Bird        Black                                44  
            Black/Brown                           6  
            Black/Chocolate                       5  
            Black/Cream                           5  
            Black/Gold                            1  
...                                             ...  
Other       White/Tricolor                       17  
            White/White                           7  
            White/Yellow                          1  
            Yellow                                1  
            Yellow/Black                          2  

[917 rows x 11 columns]

In [44]:
outcomes.pivot_table(index='Animal Type', columns='Color', aggfunc='nunique')

Age upon Outcome                                              \
Color                 Agouti Agouti/Brown Tabby Agouti/Cream Agouti/Gray   
Animal Type                                                                
Bird                     NaN                NaN          NaN         NaN   
Cat                     10.0                1.0          1.0         NaN   
Dog                      2.0                NaN          NaN         NaN   
Livestock                NaN                NaN          NaN         NaN   
Other                    3.0                NaN          NaN         1.0   

                                                                               \
Color       Agouti/White Apricot Apricot/Brown Apricot/Tricolor Apricot/White   
Animal Type                                                                     
Bird                 NaN     NaN           NaN              NaN           NaN   
Cat                  3.0     1.0           NaN              NaN           1.0   
Dog                  NaN    21.0           2.0              1.0           5.0   
Livestock            NaN     NaN           NaN              NaN           NaN   
Other                1.0     NaN           NaN              NaN           NaN   

                   ... Sex upon Outcome                                        \
Color       Black  ...     Yellow/Brown Yellow/Cream Yellow/Gray Yellow/Green   
Animal Type        ...                                                          
Bird         13.0  ...              1.0          NaN         1.0          2.0   
Cat          45.0  ...              NaN          1.0         NaN          NaN   
Dog          42.0  ...              1.0          1.0         1.0          NaN   
Livestock     3.0  ...              NaN          NaN         NaN          NaN   
Other        25.0  ...              NaN          NaN         NaN          NaN   

                                                                     \
Color       Yellow/Orange Yellow/Orange Tabby Yellow/Red Yellow/Tan   
Animal Type                                                           
Bird                  NaN                 NaN        1.0        NaN   
Cat                   3.0                 1.0        NaN        NaN   
Dog                   NaN                 NaN        NaN        2.0   
Livestock             NaN                 NaN        NaN        NaN   
Other                 NaN                 NaN        NaN        NaN   

                                        
Color       Yellow/White Yellow/Yellow  
Animal Type                             
Bird                 2.0           NaN  
Cat                  NaN           NaN  
Dog                  5.0           1.0  
Livestock            NaN           NaN  
Other                NaN           NaN  

[5 rows x 6787 columns]

In [ ]:
type_and_color[type_]

<details>
    <summary>Answer</summary>

```python
outcomes.groupby('Animal Type')['Date of Birth'].max()
```
</details>

# Pivoting a DataFrame

## `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [35]:
outcomes.drop(columns=['Calculated Age in Days 2'], inplace=True)

In [36]:
outcomes.pivot_table(index = 'Outcome Type', columns='Sex upon Outcome', aggfunc='mean')

Calculated Age in Days                             \
Sex upon Outcome          Intact Female  Intact Male Neutered Male   
Outcome Type                                                         
Adoption                     419.505568   476.596774    651.126203   
Died                         349.628049   304.392157   1859.732673   
Disposal                     407.277778   735.935484   1987.777778   
Euthanasia                  1133.424632   902.476300   2226.837838   
Missing                      266.800000   340.846154   1188.285714   
Relocate                     732.000000          NaN   1105.000000   
Return to Owner             1067.053879  1110.232182   1634.609964   
Rto-Adopt                   1481.142857  1503.175000   1254.399577   
Transfer                     409.499911   351.252856   1126.408920   

                                             
Sex upon Outcome Spayed Female      Unknown  
Outcome Type                                 
Adoption            648.050078   389.964286  
Died               2099.905263   311.192737  
Disposal           2470.166667   447.166016  
Euthanasia         2313.677966   503.333588  
Missing            1262.733333   169.250000  
Relocate            495.000000   612.200000  
Return to Owner    1751.059214   764.225275  
Rto-Adopt          1252.076923  1590.000000  
Transfer           1097.636076   169.775301

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

In [39]:
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


In [40]:
outcomes.T

,0,1,2,3,4,5,6,7,8,9,...,137087,137088,137089,137090,137091,137092,137093,137094,137095,137096
Animal ID,A794011,A776359,A821648,A720371,A674754,A659412,A814515,A689724,A680969,A840370,...,A690359,A851184,A850725,A847804,A842414,A850166,A852031,A845839,A844321,A813933
Name,Chunk,Gizmo,NaN,Moose,NaN,Princess,Quentin,*Donatello,*Zeus,Tulip,...,*Porridge,*Papaya,Monty,*Mahalia,NaN,Rainey,Noodle,*Carmen,Mia Marie,Lucille
DateTime,2019-05-08 18:20:00,2018-07-18 16:02:00,2020-08-16 11:38:00,2016-02-13 17:59:00,2014-03-18 11:47:00,2020-10-05 14:37:00,2020-05-06 07:59:00,2014-10-18 18:52:00,2014-08-05 16:59:00,2021-08-19 19:36:00,...,2022-02-26 11:34:00,2022-02-28 11:38:00,2022-02-25 11:23:00,2022-02-28 11:42:00,2021-09-17 13:38:00,2022-01-24 18:20:00,2022-02-28 12:50:00,2022-02-28 13:49:00,2022-02-28 13:04:00,2022-02-28 14:19:00
MonthYear,May 2019,Jul 2018,Aug 2020,Feb 2016,Mar 2014,Oct 2020,May 2020,Oct 2014,Aug 2014,Aug 2021,...,Feb 2022,Feb 2022,Feb 2022,Feb 2022,Sep 2021,Jan 2022,Feb 2022,Feb 2022,Feb 2022,Feb 2022
Date of Birth,2017-05-02 00:00:00,2017-07-12 00:00:00,2019-08-16 00:00:00,2015-10-08 00:00:00,2014-03-12 00:00:00,2013-03-24 00:00:00,2018-03-01 00:00:00,2014-08-01 00:00:00,2014-06-03 00:00:00,2019-08-06 00:00:00,...,2014-06-19 00:00:00,2021-02-08 00:00:00,2020-01-30 00:00:00,2011-12-08 00:00:00,2021-07-15 00:00:00,2021-11-19 00:00:00,2020-02-23 00:00:00,2020-05-05 00:00:00,2013-10-15 00:00:00,2018-12-21 00:00:00
Outcome Type,Rto-Adopt,Adoption,Euthanasia,Adoption,Transfer,Adoption,Adoption,Adoption,Adoption,Adoption,...,Transfer,Adoption,Adoption,Adoption,Adoption,Adoption,Transfer,Adoption,Adoption,Adoption
Outcome Subtype,NaN,NaN,NaN,NaN,Partner,NaN,Foster,NaN,NaN,NaN,...,Partner,NaN,NaN,NaN,NaN,NaN,Partner,Foster,Foster,NaN
Animal Type,Cat,Dog,Other,Dog,Cat,Dog,Dog,Cat,Cat,Dog,...,Dog,Cat,Dog,Cat,Dog,Cat,Dog,Dog,Dog,Dog
Sex upon Outcome,Neutered Male,Neutered Male,Unknown,Neutered Male,Intact Male,Spayed Female,Neutered Male,Neutered Male,Neutered Male,Spayed Female,...,Neutered Male,Spayed Female,Neutered Male,Spayed Female,Intact Female,Intact Male,Neutered Male,Spayed Female,Spayed Female,Spayed Female
Age upon Outcome,2 years,1 year,1 year,4 months,6 days,7 years,2 years,2 months,2 months,2 years,...,7 years,1 year,2 years,10 years,2 months,2 months,2 years,1 year,8 years,3 years


In [38]:
# Check it out!
outcomes.pivot(index='Outcome Type', columns='Sex upon Outcome',values='Calculated Age in Days')

ValueError: Index contains duplicate entries, cannot reshape

# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## `.join()`

In [45]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [46]:
toy2

,age,MP
0,63,100
1,33,200


In [47]:
# We can't just join these as they are, since we haven't specified our suffixes

toy1.join(toy2)

ValueError: columns overlap but no suffix specified: Index(['age'], dtype='object')

In [48]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


In [50]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [51]:
toy1.drop('age', axis=1).join(toy2)

,HP,age,MP
0,142,63,100
1,47,33,200


## `.merge()`

Or we could use `.merge()`:

In [53]:
toy1

,age,HP
0,63,142
1,33,47


In [54]:
toy2

,age,MP
0,63,100
1,33,200


In [52]:
toy1.merge(toy2)

,age,HP,MP
0,63,142,100
1,33,47,200


In [73]:
ds_chars = pd.read_csv('../data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [56]:
states = pd.read_csv('../data/states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


In [59]:
ds_chars.merge(states, left_on='home_state', right_on='state', how='inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [ ]:
ds_chars.merge()

## The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [95]:
outcomes.shape

(137097, 13)

In [96]:
intakes.shape

(136763, 12)

In [94]:
combined.shape

(176664, 24)

In [65]:
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX
5,william,200,VA


In [66]:
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


In [68]:
ds_chars.merge(states, left_on='home_state', right_on='state', how='outer')

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,WA,evergreen,Olympia
1,miles,200.0,WA,WA,evergreen,Olympia
2,alan,170.0,TX,TX,alamo,Austin
3,rachel,200.0,TX,TX,alamo,Austin
4,alison,300.0,DC,DC,district,Washington
5,william,200.0,VA,NaN,NaN,NaN
6,NaN,NaN,NaN,OH,buckeye,Columbus
7,NaN,NaN,NaN,OR,beaver,Salem


## `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [74]:
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [75]:
prefs = pd.read_csv('../data/preferences.csv', index_col=0)
prefs

,cuisine,genre
0,Greek,horror
1,Indian,scifi
2,American,fantasy
3,Thai,tech
4,Indian,documentary


In [76]:
ds_full = pd.concat([ds_chars, prefs])

In [77]:
ds_full

,name,HP,home_state,cuisine,genre
0,greg,200.0,WA,NaN,NaN
1,miles,200.0,WA,NaN,NaN
2,alan,170.0,TX,NaN,NaN
3,alison,300.0,DC,NaN,NaN
4,rachel,200.0,TX,NaN,NaN
0,NaN,NaN,NaN,Greek,horror
1,NaN,NaN,NaN,Indian,scifi
2,NaN,NaN,NaN,American,fantasy
3,NaN,NaN,NaN,Thai,tech
4,NaN,NaN,NaN,Indian,documentary


`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [ ]:
ds_full = pd.concat([ds_chars, prefs], axis=1)
ds_full

## Back to the Center

We have Intakes data and we have Outcomes data... time to merge!

In [80]:
# Peek at the outcomes data we already had in here
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


In [81]:
# Read in the intakes data
intakes = pd.read_csv("../data/Austin_Animal_Center_Intakes_022822.csv",
                      parse_dates=['DateTime'])
# Check it out
intakes.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,2019-01-03 16:19:00,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,2015-07-05 12:59:00,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,2016-04-14 18:43:00,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,2013-10-21 07:59:00,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,2014-06-29 10:38:00,June 2014,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray


In [86]:
# Let's try merging on Animal ID
combined = outcomes.merge(intakes, on='Animal ID', suffixes=['_outcome', '_intake'], how='inner')

In [87]:
# What was the result?
combined.head()

,Animal ID,Name_outcome,DateTime_outcome,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,...,DateTime_intake,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,Color_intake
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,2019-05-02 16:51:00,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,...,2018-07-12 12:46:00,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,...,2020-08-16 10:10:00,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [88]:
combined.shape

(176664, 24)

In [89]:
intakes.shape

(136763, 12)

In [90]:
outcomes.shape

(137097, 13)

Let's discuss/explore: did that work the way we expected?

- 

<details>
    <summary>Observation Notes</summary>

- We went from about 136k rows in each of the dataframes to 176k! Even using an inner join! Something seems off. 
    
    
</details>

In [102]:
intakes[intakes['Animal ID'] == 'A720371']

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
32813,A720371,Moose,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
90559,A720371,Moose,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [103]:
outcomes[outcomes['Animal ID'] == 'A720371']

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
9427,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,130


In [105]:
combined[combined['Animal ID'] == 'A720371']

,Animal ID,Name_outcome,DateTime_outcome,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,...,DateTime_intake,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,Color_intake
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
5,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,...,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
6,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,...,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [98]:
# We might want to try something different
# Can we clean something to make a better merge?
combined.loc[combined.duplicated(subset='Animal ID')]

,Animal ID,Name_outcome,DateTime_outcome,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,...,DateTime_intake,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,Color_intake
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
5,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,...,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
6,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,...,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
9,A659412,Princess,2020-10-05 14:37:00,Oct 2020,2013-03-24,Adoption,NaN,Dog,Spayed Female,7 years,...,2020-10-05 11:20:00,October 2020,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,7 years,Chihuahua Shorthair Mix,Brown
10,A659412,Princess,2018-06-15 11:37:00,Jun 2018,2013-03-24,Adoption,NaN,Dog,Spayed Female,5 years,...,2018-06-13 12:55:00,June 2018,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,5 years,Chihuahua Shorthair Mix,Brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176642,A850561,Max,2022-01-27 18:05:00,Jan 2022,2020-09-27,Adoption,NaN,Dog,Neutered Male,1 year,...,2022-01-26 17:31:00,January 2022,Austin (TX),Public Assist,Normal,Dog,Neutered Male,1 year,Belgian Malinois/German Shepherd,Black/Brown
176643,A850561,Max,2022-02-05 16:22:00,Feb 2022,2020-09-27,Adoption,NaN,Dog,Neutered Male,1 year,...,2022-01-31 17:22:00,January 2022,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,1 year,Belgian Malinois/German Shepherd,Black/Brown
176644,A850561,Max,2022-02-05 16:22:00,Feb 2022,2020-09-27,Adoption,NaN,Dog,Neutered Male,1 year,...,2022-02-27 16:52:00,February 2022,1414 South Lamar Boulevard in Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,1 year,Belgian Malinois/German Shepherd,Black/Brown
176645,A850561,Max,2022-02-05 16:22:00,Feb 2022,2020-09-27,Adoption,NaN,Dog,Neutered Male,1 year,...,2022-01-26 17:31:00,January 2022,Austin (TX),Public Assist,Normal,Dog,Neutered Male,1 year,Belgian Malinois/German Shepherd,Black/Brown


In [106]:
# Try again
clean_intakes = intakes.drop_duplicates(subset=['Animal ID'])
clean_outcomes = outcomes.drop_duplicates(subset=['Animal ID'])

In [107]:
clean_intakes[clean_intakes['Animal ID'] == 'A720371']

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
32813,A720371,Moose,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [108]:
clean_combined_df = clean_intakes.merge(clean_outcomes, on='Animal ID', how='inner', suffixes=['_intake', '_outcome'])

In [109]:
clean_combined_df.head()

,Animal ID,Name_intake,DateTime_intake,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,...,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,Breed_outcome,Color_outcome,Calculated Age in Days
0,A786884,*Brock,2019-01-03 16:19:00,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,...,Jan 2019,2017-01-03,Transfer,Partner,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,735
1,A706918,Belle,2015-07-05 12:59:00,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,...,Jul 2015,2007-07-05,Return to Owner,NaN,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,2922
2,A724273,Runster,2016-04-14 18:43:00,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,...,Apr 2016,2015-04-17,Return to Owner,NaN,Dog,Neutered Male,1 year,Basenji Mix,Sable/White,370
3,A665644,NaN,2013-10-21 07:59:00,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,...,Oct 2013,2013-09-21,Transfer,Partner,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,30
4,A682524,Rio,2014-06-29 10:38:00,June 2014,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,...,Jul 2014,2010-06-29,Return to Owner,NaN,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,1464


In [110]:
clean_combined_df[clean_combined_df['Animal ID'] == 'A720371']

,Animal ID,Name_intake,DateTime_intake,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,...,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,Breed_outcome,Color_outcome,Calculated Age in Days
31721,A720371,Moose,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,...,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128


# Level Up: Quick Column Name Clean Up Code

Throwing a quick use of a lambda function your way:

In [ ]:
outcomes_renamed = outcomes.rename(columns = lambda x: x.replace(" ", "_").lower())
outcomes_renamed.head()

# Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

## Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [ ]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

Then we can use:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

## See More Rows

Or suppose we want `pandas` to show more rows.

In [ ]:
df2 = pd.DataFrame(np.array(range(100)))
df2

In that case we can use:

In [ ]:
pd.set_option('display.max_rows', 100)

df2